# Browser/Kernel data interchange

Jupyter is a HTML/Javascript/jquery applicaiton which runs a programming language in the background, and renders the results on a browser page.

This examples show how to pass variables between the dynamic page and the kernel, through the introspective JS "Jupyter" API.

### Sending a variable from Javascript to the Kernel

The following code creates a variable "browser_side" on the browser page youre running, and then asks the kernel to execute a statement.

Incidentally, the kernel is python, and we can assign a json-formatted object directly into a python variable.

In [1]:
%%javascript
var obj = {
    name: "an object",
    value: 10.1
};
var data = JSON.stringify(obj);
/* This will create kernel_obj on the kernel, 
   with the same content as 'obj' */
Jupyter.notebook.kernel.execute("kernel_obj = " + data);

<IPython.core.display.Javascript object>

You can then see that, kernel_obj is now known to the remote python kernel:

In [2]:
print("I am '{0}' and, my value is {1}".format(
    kernel_obj['name'], kernel_obj['value']))

I am 'an object' and, my value is 10.1


Also interestingly, we can just ask the kernel
to evaluate the variable 'as is', and get a textual description
that is rendered on the page as "execution result".


In [3]:
kernel_obj

{'name': 'an object', 'value': 10.1}

This is achieved by the kernel handler receving an `execution_result` message, which then is rendedered as `Out[..]:` by the javascript engine on the page.

Knowing this, we can now retreive a value from the kernel.

### Reading a variable from the kernel into the browser Javascript

For the inverse procedure, we need to ask the kernel to evaluate the variable, and send its text representation back to us.

The `Jupyter.notebook.kernel` object has a `execute` method that can be provided with various callbacks, through an object passed as a paramter. We are interested in the `execution_result` message, which is sent to the `iopub.output` callback.

In [4]:
%%javascript
function display(msg){
    /* 'element' reflects the output of this cell in Jquery */
    console.log(msg)
    element.text("Receivced '"+ msg.msg_type + 
                 "' with content '" +
                 msg.content.data['text/plain'] +"'");
};

Jupyter.notebook.kernel.execute(
       "kernel_obj",   /* Evaluating the variable symbol */
       {iopub:{output: display}}, /* output message callback */
       {silent:false}); /* Ask to render the variable evaluation */


<IPython.core.display.Javascript object>

With this method, it's possible to create data on the browser, send it to the kernel for handling and storage, and get it back on the browser for dynamic rendering.